In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import datasets, layers, models, losses, Model
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
df1 = pd.read_excel('wclassesFiltered.xlsx')


In [ ]:
train = df1.sample(frac=0.8,random_state=200)
test = df1.drop(train.index)

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  r'C:\Users\biamc\Documents\Insper\Machine Learning\SArt',
  labels='inferred',
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(256, 256),
  batch_size=32)

In [ ]:
target_size=(256,256) # set the size of the images
color_mode='rgb' # set the type of image
class_mode= 'categorical' # set the class mode
batch_size=32  # set the batch size
subset='training' # set to 'training', or 'valiatiom' or leave as None

train_gen = ImageDataGenerator()
test_gen = ImageDataGenerator()
train_imgs= train_gen.flow_from_dataframe(dataframe=train, 
          x_col='file',
          y_col=['style'],
          target_size=target_size,color_mode=color_mode,
          class_mode='raw', batch_size=batch_size,shuffle=True, seed=123,
          subset='training')
test_imgs= test_gen.flow_from_dataframe(dataframe=test, 
          x_col='file',
          y_col=['style'],
          target_size=target_size,color_mode=color_mode,
          class_mode='raw', batch_size=batch_size,shuffle=True, seed=123,
          subset='training')

In [ ]:
base_model = tf.keras.applications.ResNet152(weights = 'imagenet', include_top = False, input_shape = (32,32,3))
for layer in base_model.layers:
  layer.trainable = False

In [ ]:
x = layers.Flatten()(base_model.output)
x = layers.Dense(1000, activation='relu')(x)
predictions = layers.Dense(10, activation = 'softmax')(x)

In [ ]:
head_model = Model(inputs = base_model.input, outputs = predictions)
head_model.compile(optimizer='adam', loss=losses.sparse_categorical_crossentropy, metrics=['accuracy'])

In [ ]:
history = head_model.fit(x_train, y_train, batch_size=64, epochs=40, validation_data=(x_val, y_val))

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(15,15))

axs[0].plot(history.history['loss'])
axs[0].plot(history.history['val_loss'])
axs[0].title.set_text('Training Loss vs Validation Loss')
axs[0].set_xlabel('Epochs')
axs[0].set_ylabel('Loss')
axs[0].legend(['Train','Val'])

axs[1].plot(history.history['accuracy'])
axs[1].plot(history.history['val_accuracy'])
axs[1].title.set_text('Training Accuracy vs Validation Accuracy')
axs[1].set_xlabel('Epochs')
axs[1].set_ylabel('Accuracy')
axs[1].legend(['Train', 'Val'])

In [ ]:
head_model.evaluate(x_test, y_test)